In [1]:
import pandas as pd
import numpy as np
from hybrid_mdmc.functions import get_PSSrxns,scalerxns

def pure_kmc_reaction_selection(probabilities, reaction_IDs):
    u2 = 0
    while u2 == 0:
        u2 = np.random.random()
    dt = -np.log(u2)/np.sum(probabilities)
    u1 = np.random.random()
    rxn_idx = np.argwhere(np.cumsum(probabilities)>=np.sum(probabilities)*u1)[0][0]
    selected_reaction_ID = reaction_IDs[rxn_idx]

    return selected_reaction_ID, dt

In [3]:
windowsize_slope = 800
scalingcriteria_concentration_slope = 2.0
scalingcriteria_concentration_cycles = 1

windowsize_rxnselection = 48
scalingcriteria_rxnselection_count = 1

windowsize_scalingpause = 5

rates = {
    1: 2e7, 2: 2e7, 3: 2e7, 4: 2e7,
    5: 1e2,
    6: 2e3,
}
scalingfactor_adjuster = rates[6]/rates[1]
scalingfactor_minimum = scalingfactor_adjuster

progression = pd.DataFrame({
    'time': [0],
    'A': [200],
    'B': [0],
    'C': [0],
    'D': [0],
    0: [0],
    1: [0],
    2: [0],
    3: [0],
    4: [0],
    5: [0],
    6: [0],
},
index=[0])

reaction_scaling = pd.DataFrame({
    1: [1.0],
    2: [1.0],
    3: [1.0],
    4: [1.0],
    5: [1.0],
    6: [1.0],
},
index=[0])

rxnmatrix = pd.DataFrame({
    'A': [-1, 1, 0, 0, 0, 0],
    'B': [1, -1, 1, -1, 0, 1],
    'C': [0, 0, -1, 1, 0, 0],
    'D': [0, 0, 0, 0, 1, -1],
},
index=[1,2,3,4,5,6])

species = ['A', 'B', 'C', 'D']
reaction_IDs = [1, 2, 3, 4, 5, 6]
reactants = {1: 'A', 2: 'B', 3: 'C', 4: 'D', 5: 'A', 6: 'B'}
products = {1: 'B', 2: 'A', 3: 'D', 4: 'C', 5: 'C', 6: 'D'}

for step in range(300):

    PSSrxns = get_PSSrxns(
        rxnmatrix,
        reaction_scaling,
        progression,
        windowsize_slope,
        windowsize_rxnselection,
        scalingcriteria_concentration_slope,
        scalingcriteria_concentration_cycles,
        scalingcriteria_rxnselection_count,
    )

    reaction_scaling = scalerxns(
        reaction_scaling,
        PSSrxns,
        windowsize_scalingpause,
        scalingfactor_adjuster,
        scalingfactor_minimum,
        rxnlist='all',
    )

    probabilities = np.array([rates[_]*progression.loc[step,reactants[_]]*reaction_scaling.loc[step+1,_] for _ in reaction_IDs])
    selected_reaction_ID, dt = pure_kmc_reaction_selection(probabilities, reaction_IDs)
    progression.loc[step+1] = [progression.loc[step,'time']+dt] + [progression.loc[step,_] for _ in species] + [0] + [0 for _ in reaction_IDs]
    progression.loc[step+1,reactants[selected_reaction_ID]] -= 1
    progression.loc[step+1,products[selected_reaction_ID]] += 1
    progression.loc[step+1,selected_reaction_ID] += 1

In [4]:
progression

,time,A,B,C,D,0,1,2,3,4,5,6
0,0.000000e+00,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.211983e-10,199.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,6.117151e-10,198.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,6.703602e-10,197.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,6.801702e-10,196.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
296,7.119264e-08,98.0,102.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
297,7.119542e-08,97.0,103.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
298,7.141553e-08,96.0,104.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
299,7.209741e-08,95.0,105.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
